In [5]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
#ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-135742
Azure region: southcentralus
Subscription id: a0a76bad-11a1-4a2d-9887-97a29122c8ed
Resource group: aml-quickstarts-135742


In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Choose a name for your CPU cluster
cpu_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)
print(compute_target.get_status().serialize())

Found existing cluster, use it.

Running
{'errors': [], 'creationTime': '2021-01-23T03:51:16.068721+00:00', 'createdBy': {'userObjectId': 'f45985be-4ead-48df-a064-6f3ac3156482', 'userTenantId': '660b3398-b80e-49d2-bc5b-ac1dc93b5254', 'userName': 'ODL_User 135742'}, 'modifiedTime': '2021-01-23T03:53:47.730243+00:00', 'state': 'Running', 'vmSize': 'STANDARD_DS3_V2'}


In [10]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,normal
from azureml.core import ScriptRunConfig
from azureml.core import Environment
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        'learning_rate': normal(10,3),
        'keep_probability': uniform(0.05,0.1)
    }
)

# Specify a Policy
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

#if "training" not in os.listdir():
#    os.mkdir("./training")
project_folder = './'


#sklearn_env = Environment.from_conda_specification(name = 'sklearn-env', file_path = './conda_dependencies.yml')

# Create a SKLearn estimator for use with train.py
compute_target = ws.compute_targets['auto-ml']
est = SKLearn(source_directory=project_folder,
            entry_script='train.py',
            compute_target=compute_target,
            #arguments=['--kernel', 'linear', '--penalty', 1.0],
            #environment=sklearn_env)
            )

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est, 
                                    hyperparameter_sampling=ps,
                                    policy=early_termination_policy,
                                    primary_metric_name='Accuracy', 
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                                    max_total_runs=8,
                                    max_concurrent_runs=4)

In [11]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)
assert(hyperdrive_run.get_status() == "Completed")

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_9bc63bc5-6746-4373-b92a-01d581205001
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_9bc63bc5-6746-4373-b92a-01d581205001?wsid=/subscriptions/a0a76bad-11a1-4a2d-9887-97a29122c8ed/resourcegroups/aml-quickstarts-135742/workspaces/quick-starts-ws-135742

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-01-23T02:54:08.438955][API][INFO]Experiment created<END>\n""<START>[2021-01-23T02:54:09.287438][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n"<START>[2021-01-23T02:54:09.4657475Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2021-01-23T02:54:09.606402][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_9bc63bc5-6746-4373-b92a-01d581205001
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_9bc63bc5-6746-4373-b92a-01d581205001?wsid=/subscriptions/a0a76

In [12]:
import joblib
# Get your best run and save the model from that run.|

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])
print(best_run.get_file_names())
model = best_run.register_model(model_name='hyperdrive-train', model_path='outputs/model.joblib')

['--keep_probability', '0.05106288981937711', '--learning_rate', '18.58767652434821']
['azureml-logs/55_azureml-execution-tvmps_f58d7b9aafce8c94e152515bc25d82acd4250f95d5b9d1f3049ad35952b89b19_d.txt', 'azureml-logs/65_job_prep-tvmps_f58d7b9aafce8c94e152515bc25d82acd4250f95d5b9d1f3049ad35952b89b19_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_f58d7b9aafce8c94e152515bc25d82acd4250f95d5b9d1f3049ad35952b89b19_d.txt', 'logs/azureml/95_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/engine_spans_l_b81383ce-7dc8-4c9d-a090-404e897c5979.jsonl', 'logs/azureml/dataprep/python_span_l_b81383ce-7dc8-4c9d-a090-404e897c5979.jsonl', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model.joblib']


In [7]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

web_path = ['https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv']
ds = TabularDatasetFactory.from_delimited_files(path=web_path)

In [8]:
from train import clean_data
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x['y'] = y

In [9]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="regression",
    primary_metric='normalized_root_mean_squared_error',
    training_data=x,
    label_column_name='y',
    n_cross_validations=5)

In [10]:
# Submit your automl run

exp = Experiment(ws,"automl_run")
automl_run = exp.submit(config=automl_config, show_output = True)

No run_configuration provided, running on local with default configuration
Running on local machine
Parent Run ID: AutoML_a0da03bd-7e12-47c3-8ddb-15d982f9006f



In [21]:
from azureml.automl.runtime.onnx_convert import *

# Get your best run and save the model from that run.

best_run, onxx_model = automl_run.get_output(return_onxx_model = True)
print(best_run)
print(onxx_model)
onxx_fl_path = "./best_model.onxx"
OnxxConverter.save_onxx_model(onxx_model, onxx_fl_path)

ImportError: cannot import name 'OnxxConverter'